# Using the GridR's Grid Resampling Chain - Basic

This guide demonstrates how to effectively use GridR's `basic_grid_resampling_chain` that wraps the core `array_grid_resampling` method. It covers key aspects of its operation, helping you understand how to manage your data and resources efficiently.

Here's what we'll explore:
- **Managing I/O Datasets**: How to properly handle input and output datasets when working with the chaining function.
- **Memory Resource Management**: Techniques for controlling memory usage during computations.
- **Extended Computational Features**: A comparison of the chain's capabilities versus the core array_grid_resampling method.

First, let's address "why basic"? This chain is prefixed "basic" because it provides users with direct control over several memory usage-related parameters, allowing for fine-tuned resource management. Currently, this memory management is not automatic, requiring users to adapt these parameters for different use cases; automatic management is identified as a future enhancement for improved usability.


## Setting things up

In [ ]:
import os
import sys

import numpy as np
from notebook_utils import plot_im, mpl_plot_wrapper

sys.path.insert(0, "/".join(["..","python"]))

IN_DOC_BUILD = os.environ.get("DOC_BUILD", "0") == "1"
if not IN_DOC_BUILD:
    from bokeh.io import output_notebook # enables plot interface in J notebook
    output_notebook()

First proceed with some import : standard, community tiers and grid packages.

We also import the well known mandrill raster image.

In [ ]:
import os
from pathlib import Path
import tempfile

import numpy as np
import shapely
import rasterio
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
    
from gridr.io.common import GridRIOMode, safe_raster_open
from gridr.chain.grid_resampling_chain import (basic_grid_resampling_chain,
        GEOMETRY_RASTERIZE_KWARGS)

from gridr.misc.mandrill import mandrill

In order to work with the `basic_grid_resampling_chain` we will need at least :
- a raster image to read
- a resampling grid to read

First we are going to generate the raster image from our 3 channels mandrill image.

In [ ]:
RASTER_IN = './grid_resampling_chain_001_raster_in.tif'
GRID_IN_F64 = './grid_resampling_chain_001_grid_in_f64.tif'

In [ ]:
def write_array(array, dtype, fileout):
    if array.ndim == 3:
        with rasterio.open(fileout, "w", driver="GTiff", dtype=dtype,
                height=array.shape[1], width=array.shape[2], count=array.shape[0],
                ) as array_in_ds:
            for band in range(array.shape[0]):
                array_in_ds.write(array[band].astype(dtype), band+1)
            array_in_ds = None
    elif array.ndim == 2:
        with rasterio.open(fileout, "w", driver="GTiff", dtype=dtype,
                height=array.shape[0], width=array.shape[1], count=1,
                ) as array_in_ds:
            array_in_ds.write(array.astype(dtype), 1)
            array_in_ds = None

def shape2(array):
    if array.ndim == 3:
        return (array.shape[1], array.shape[2])
    else:
        return array.shape

# write mandrill as tif
write_array(mandrill, dtype=mandrill.dtype, fileout=RASTER_IN)

In [ ]:
def create_grid(nrow, ncol, origin_pos, origin_node, v_row_y, v_row_x, v_col_y, v_col_x, dtype):
    """
    """
    x = np.arange(0, ncol, dtype=dtype)
    y = np.arange(0, nrow, dtype=dtype)
    xx, yy = np.meshgrid(x, y)
    xx -= origin_pos[0]
    yy -= origin_pos[1]
    yyy = origin_node[0] + yy * v_row_y + xx * v_col_y
    xxx = origin_node[1] + yy * v_row_x + xx * v_col_x
    return yyy, xxx

In [ ]:
nrow = 50
ncol = 40
origin_pos = np.array((0.3,0.2))
origin_node = np.array((0., 0.))
v_row_y = 5.2
v_row_x = 1.2
v_col_y = -2.7
v_col_x = 7.1
grid_row_f64, grid_col_f64 = create_grid(nrow, ncol, origin_pos, origin_node, v_row_y, v_row_x, v_col_y, v_col_x, dtype=np.float64)

write_array(np.array([grid_row_f64, grid_col_f64]), dtype=np.float64, fileout=GRID_IN_F64)

In [ ]:
from typing import Optional, Tuple
import matplotlib
plt.style.use('ggplot')
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

@mpl_plot_wrapper
def plot_grid_on_image(
        z: int,
        grid_row: np.ndarray, 
        grid_col: np.ndarray, 
        grid_resolution: Tuple[int, int],
        array_shape: Tuple[int, int], 
        mask: Optional[np.ndarray] = None, 
        win: Optional[np.ndarray] = None,
        raster_image: Optional[np.ndarray] = None,
        raster_image_mask: Optional[np.ndarray] = None,
        prefix: Optional[str] = None,
    ):
    """
    Parameters
    ----------
    grid_row : np.ndarray
        Array of row coordinates for each grid point.
    grid_col : np.ndarray
        Array of column coordinates for each grid point.
    grid_resolution : Tuple[int, int]
        The resolution of the grid as a tuple (row_resolution, col_resolution).
        Only used if `win` is not None.
    array_shape : Tuple[int, int]
        The shape of the array (height, width) that the grid is being applied on.
    mask : Optional[np.ndarray], optional
        A binary mask to apply to the grid points, by default None.
    win : Optional[np.ndarray], optional
        A window (in GridR's convention) defining the production window, by default None.
    raster_image : Optional[np.ndarray], optional
        A 2D NumPy array representing the raster image to display as the
        background, by default None.
    prefix : Optional[str], optional
        A prefix to add to the plot's title, by default None.

    Returns
    -------
    matplotlib.pyplot.figure
        The created matplotlib figure
    """
    colors = {
        "blue": "dodgerblue",
        "red": "crimson",
        "orange": "darkorange",
        "grey": "lightsteelblue",
        "green": "limegreen",
        "purple": "mediumslateblue",
        "cyan": "deepskyblue",
    }

    height_px, width_px = int(z*array_shape[0]), int(z*array_shape[1])
    dpi = 100
    fig_width_in = width_px / dpi
    fig_height_in = height_px / dpi
    
    fig = plt.figure(figsize=(fig_width_in, fig_height_in), dpi=dpi)
    ax = fig.add_subplot(111)
    alpha_grid_lines = 0.4
    alpha_grid_points = 1.
    
        # Afficher l'image raster en arrière-plan si fournie
    if raster_image is not None:
        # L'étendue (extent) de l'image est cruciale pour la faire correspondre aux coordonnées.
        # [xmin, xmax, ymin, ymax]
        # Ici, x correspond aux colonnes (grid_col) et y aux lignes (grid_row)
        # plt.gca().invert_yaxis() est utilisé plus tard, donc ymax sera la première ligne (0)
        # et ymin sera la dernière ligne (array_shape[0] - 1)
        ax.imshow(raster_image, cmap='gray', alpha=0.3,
                   extent=[0, width_px, height_px, 0]) # extent = [left, right, bottom, top]
                                                                  # Notez l'ordre pour y : bottom (max row) puis top (min row)
                                                                  # en raison de invert_yaxis()
                                                                  # C'est parce que imshow affiche l'origine en haut à gauche par défaut.

    if raster_image_mask is not None:
        invalid_color = (0.8627, 0.0784, 0.2353, 0.5)
        mask_rgba = np.zeros((*raster_image_mask.shape, 4), dtype=np.float32)
        mask_rgba[raster_image_mask == 0] = invalid_color
        ax.imshow(mask_rgba,
              extent=[0, width_px, height_px, 0],
              interpolation='nearest')        
    
    if win is not None:
        target_win, _ = oversample_regular_grid(
            grid = np.array((grid_row, grid_col)),
            grid_oversampling_row = grid_resolution[0],
            grid_oversampling_col = grid_resolution[1],
            grid_mask = None,
            win = win,
            )
        
        top = list(zip(target_win[0][0,:], target_win[1][0,:]))
        right = list(zip(target_win[0][:,-1], target_win[1][:,-1]))
        bottom = list(zip(target_win[0][-1,:][::-1], target_win[1][-1,:][::-1]))
        left = list(zip(target_win[0][:,0][::-1], target_win[1][:,0][::-1]))
        win_contour = top + right + bottom + left
        ax.plot([v[1] for v in win_contour], [v[0] for v in win_contour], linestyle='--', linewidth=1., color=colors['green'])
    
    # Afficher les lignes horizontales
    for i in range(grid_row.shape[0]):
        ax.plot(grid_col[i], grid_row[i], color=colors["grey"], linestyle='-', alpha=alpha_grid_lines)
    
    # Afficher les lignes verticales
    for j in range(grid_row.shape[1]):
         ax.plot(grid_col[:,j], grid_row[:,j], color=colors["grey"], linestyle='-', alpha=alpha_grid_lines)
    

    if mask is not None:
        masked_index = np.where(grid_mask==0)
        out_of_bounds_index = np.where(np.logical_or(
                         np.logical_or(grid_row < 0., grid_row > array_shape[0] - 1.),
                         np.logical_or(grid_col < 0., grid_col > array_shape[1] - 1.)
                        ))
        non_masked_index = np.where(np.logical_and(grid_mask==1,
                                                   ~np.logical_or(
                         np.logical_or(grid_row < 0., grid_row > array_shape[0] - 1.),
                         np.logical_or(grid_col < 0., grid_col > array_shape[1] - 1.)
                        )))
        
        ax.scatter(grid_col[masked_index].reshape(-1), grid_row[masked_index].reshape(-1), color=colors['red'], s=z*8, alpha=alpha_grid_points, edgecolor='black', linewidth=0.1)
        ax.scatter(grid_col[out_of_bounds_index].reshape(-1), grid_row[out_of_bounds_index].reshape(-1), color=colors['orange'], s=z*8, alpha=0.9, edgecolor='black', linewidth=0.1)
        ax.scatter(grid_col[non_masked_index].reshape(-1), grid_row[non_masked_index].reshape(-1), color=colors['blue'], s=z*8, alpha=alpha_grid_points, edgecolor='black', linewidth=0.1)
    else:
        ax.scatter(grid_col.reshape(-1), grid_row.reshape(-1), color=colors['blue'], s=z*6, alpha=alpha_grid_points, edgecolor='darkblue', linewidth=0.1)

    # Ajouter des labels pour les axes
    ax.set_xlabel('Columns', fontsize=8)
    ax.set_ylabel('Rows', fontsize=8)
    
    # Ajuster l'axe des X et des Y pour mieux voir le quadrillage
    ax.set_xlim(np.min(grid_col) - 10, np.max(grid_col) + 10)
    ax.set_ylim(np.min(grid_row) - 10, np.max(grid_row) + 10)
    
    # Ajouter un titre
    if prefix is not None:
        ax.set_title(prefix)
    
    # Afficher le quadrillage
    ax.grid(False)
    
    # Inverser l'axe des Y et définir l'aspect
    ax.invert_yaxis()
    ax.set_aspect('equal', adjustable='box')

    return fig

In [ ]:
plot_grid_on_image(1.2, grid_row_f64, grid_col_f64, (1, 1), (mandrill.shape[1], mandrill.shape[2]), None, None, mandrill[0], None, prefix=None)

## I/O Datasets

The GridR's "chain"-called methods work on rasterio DatasetReader and DatasetWriter objets. We need to create a context with all the required datasets.

As far as the inputs are concerned, its quite easy : we just have to call open on read mode.

Concerning the output rasters, its quite more tedious : we have to define opening arguments such as the `height`, `width`, `count`, `driver`, `dtype`, ...
Let's start simple :
- We are going to apply the grid transform with a (1, 1) resolution with no other arguments (no window, no mask related options, ...).
- We will use the Geotiff format with a float64 data type.
- We will work on the first band only

Given a (1, 1) resolution, the output raster's shape corresponds to the grid's shape.

The output raster opening arguments can be defined as following :

In [ ]:
output_shape = grid_row_f64.shape

raster_out_open_args = {
    'driver': "GTiff",
    'dtype': np.float64,
    'height': output_shape[0],
    'width': output_shape[1],
    'count': 1
}

In [ ]:
output_raster_path = "./grid_resampling_chain_001_output_raster.tif"

with rasterio.open(GRID_IN_F64, 'r') as grid_in_ds, \
        rasterio.open(RASTER_IN, 'r') as array_src_ds, \
        rasterio.open(output_raster_path, "w", **raster_out_open_args) as array_out_ds:

    basic_grid_resampling_chain(
            grid_ds = grid_in_ds,
            grid_row_coords_band = 1,
            grid_col_coords_band = 2,
            grid_resolution = (1, 1),
            array_src_ds = array_src_ds,
            array_src_bands = 1,
            array_out_ds = array_out_ds,
            interp = "cubic",
            nodata_out = 0,
        )

Let's open the output and show the image.

In [ ]:
with rasterio.open(output_raster_path, "r") as ds:
    print(ds.profile) 
    plot_im(ds.read(1))

That's a tiny image result due to the subsampling the grid is performing !

Nevertheless, we can notice that the `nodata_out` value has been set where input image was not available.

Let's apply a zoom by increasing the resolution to be (10, 10).

Now it's interesting : we have to give to rasterio the output shape for that resolution. It is not quite complicated to compute from scratch but we will use here the `grid_full_resolution_shape` utility method.

In [ ]:
from gridr.core.grid.grid_commons import grid_full_resolution_shape

grid_resolution = (8, 8)
output_shape = grid_full_resolution_shape(shape=grid_row_f64.shape, resolution=grid_resolution)

print(f"output shape : {output_shape}")

In [ ]:
raster_out_open_args = {
    'driver': "GTiff",
    'dtype': np.float64,
    'height': output_shape[0],
    'width': output_shape[1],
    'count': 1
}

# We overwrite on the previous output raster - no need to keep it
with rasterio.open(GRID_IN_F64, 'r') as grid_in_ds, \
        rasterio.open(RASTER_IN, 'r') as array_src_ds, \
        rasterio.open(output_raster_path, "w", **raster_out_open_args) as array_out_ds:

    basic_grid_resampling_chain(
            grid_ds = grid_in_ds,
            grid_row_coords_band = 1,
            grid_col_coords_band = 2,
            grid_resolution = grid_resolution,
            array_src_ds = array_src_ds,
            array_src_bands = 1,
            array_out_ds = array_out_ds,
            interp = "cubic",
            nodata_out = 0,
        )

In [ ]:
with rasterio.open(output_raster_path, "r") as ds:
    print(ds.profile) 
    plot_im(ds.read(1))

Some data have been set to `nodata_out`. It is possible to generate a binary output validity mask.

Such a mask will be compliant with GridR's validity convention (1 is valid, 0 is no valid).

In order to do that we have to pass a dedicated output raster for mask.

In [ ]:
mask_out_open_args = {
    'driver': "GTiff",
    'dtype': np.uint8, # <= save as unsigned int
    'height': output_shape[0],
    'width': output_shape[1],
    'count': 1, # <= the mask is common for all bands
    'nbits': 1, # <= GDAL option to save as true binary for less disk usage
}

output_mask_path = "./grid_resampling_chain_001_output_mask.tif"

# We overwrite on the previous output raster - no need to keep it
with rasterio.open(GRID_IN_F64, 'r') as grid_in_ds, \
        rasterio.open(RASTER_IN, 'r') as array_src_ds, \
        rasterio.open(output_raster_path, "w", **raster_out_open_args) as array_out_ds, \
        rasterio.open(output_mask_path, "w", **mask_out_open_args) as mask_out_ds:

    basic_grid_resampling_chain(
            grid_ds = grid_in_ds,
            grid_row_coords_band = 1,
            grid_col_coords_band = 2,
            grid_resolution = grid_resolution,
            array_src_ds = array_src_ds,
            array_src_bands = 1,
            array_out_ds = array_out_ds,
            interp = "cubic",
            nodata_out = 0,
            mask_out_ds = mask_out_ds,
        )

In [ ]:
with rasterio.open(output_raster_path, "r") as raster_ds, \
        rasterio.open(output_mask_path, "r") as mask_ds:
    print(mask_ds.profile) 
    plot_im({"output image": raster_ds.read(1), "output mask": mask_ds.read(1)})

### Using a raster mask for the grid

In [ ]:
grid_mask_in_path = './grid_resampling_chain_001_grid_mask_in_u8_1_0.tif'

grid_mask_in_band = 1
grid_mask_in_valid_value = 1
grid_mask_in_invalid_value = 0
grid_mask_dtype = np.uint8

roi = np.array(((10,40), (5,100)))
grid_mask = np.full(grid_row_f64.shape, grid_mask_in_valid_value, dtype=np.uint8)
grid_mask[np.logical_and(
        np.logical_and(grid_row_f64 >= roi[0][0], grid_row_f64 <= roi[0][1]),
        np.logical_and(grid_col_f64 >= roi[1][0], grid_col_f64 <= roi[1][1]))] = grid_mask_in_invalid_value

write_array(grid_mask, dtype=grid_mask_dtype, fileout=grid_mask_in_path)

In [ ]:
plot_grid_on_image(1.2, grid_row_f64, grid_col_f64, (10, 10), (mandrill.shape[1], mandrill.shape[2]),
                   mask=grid_mask, win=None, raster_image=mandrill[0], prefix=None)

In [ ]:
# We overwrite on the previous output raster - no need to keep it
with rasterio.open(GRID_IN_F64, 'r') as grid_in_ds, \
        rasterio.open(RASTER_IN, 'r') as array_src_ds, \
        rasterio.open(grid_mask_in_path, "r") as grid_mask_in_ds, \
        rasterio.open(output_raster_path, "w", **raster_out_open_args) as array_out_ds, \
        rasterio.open(output_mask_path, "w", **mask_out_open_args) as mask_out_ds:

    basic_grid_resampling_chain(
            grid_ds = grid_in_ds,
            grid_row_coords_band = 1,
            grid_col_coords_band = 2,
            grid_resolution = grid_resolution,
            array_src_ds = array_src_ds,
            array_src_bands = 1,
            array_out_ds = array_out_ds,
            interp = "cubic",
            nodata_out = 400, # <= just to illustrate
            mask_out_ds = mask_out_ds,
        
            grid_mask_in_ds = grid_mask_in_ds,
            grid_mask_in_unmasked_value = grid_mask_in_valid_value, # <= give the validity value
            grid_mask_in_band = 1, # <= give the band index
        )

In [ ]:
with rasterio.open(output_raster_path, "r") as raster_ds, \
        rasterio.open(output_mask_path, "r") as mask_ds:
    print(mask_ds.profile) 
    plot_im({"output image": raster_ds.read(1), "output mask": mask_ds.read(1)})

#### Using different mask type and convention

In [ ]:
grid_mask_in_path_i8 = './grid_resampling_chain_001_grid_mask_in_i8_0_m10.tif'

grid_mask_in_band_i8 = 1
grid_mask_in_valid_value_i8 = 0
grid_mask_in_invalid_value_i8 = -10
grid_mask_dtype_i8 = np.int8

roi = np.array(((10,40), (5,100)))
grid_mask_i8 = np.full(grid_row_f64.shape, grid_mask_in_valid_value_i8, dtype=grid_mask_dtype_i8)
grid_mask_i8[np.logical_and(
        np.logical_and(grid_row_f64 >= roi[0][0], grid_row_f64 <= roi[0][1]),
        np.logical_and(grid_col_f64 >= roi[1][0], grid_col_f64 <= roi[1][1]))] = grid_mask_in_invalid_value_i8

write_array(grid_mask_i8, dtype=grid_mask_dtype_i8, fileout=grid_mask_in_path_i8)

with rasterio.open(GRID_IN_F64, 'r') as grid_in_ds, \
        rasterio.open(RASTER_IN, 'r') as array_src_ds, \
        rasterio.open(grid_mask_in_path_i8, "r") as grid_mask_in_ds, \
        rasterio.open(output_raster_path, "w", **raster_out_open_args) as array_out_ds, \
        rasterio.open(output_mask_path, "w", **mask_out_open_args) as mask_out_ds:

    basic_grid_resampling_chain(
            grid_ds = grid_in_ds,
            grid_row_coords_band = 1,
            grid_col_coords_band = 2,
            grid_resolution = grid_resolution,
            array_src_ds = array_src_ds,
            array_src_bands = 1,
            array_out_ds = array_out_ds,
            interp = "cubic",
            nodata_out = 0,
            mask_out_ds = mask_out_ds,
        
            grid_mask_in_ds = grid_mask_in_ds,
            grid_mask_in_unmasked_value = grid_mask_in_valid_value_i8, # <= give the validity value
            grid_mask_in_band = 1, # <= give the band index
        )

### Using a raster mask for the image

In [ ]:
array_src_mask_validity_valid = 1
array_src_mask_validity_invalid = 0

array_in_mask = np.full(mandrill[0].shape, array_src_mask_validity_valid, dtype=np.uint8)
masked_pos = slice(50,71), slice(150, 171) 
array_in_mask[masked_pos] = array_src_mask_validity_invalid

raster_mask_in_path_u8 = './grid_resampling_chain_001_raster_mask_in_u8_1_0.tif'

write_array(array_in_mask, dtype=np.uint8, fileout=raster_mask_in_path_u8)


In [ ]:
plot_grid_on_image(1.2, grid_row_f64, grid_col_f64, (10, 10), (mandrill.shape[1], mandrill.shape[2]),
                   mask=grid_mask, win=None, raster_image=mandrill[0], raster_image_mask=array_in_mask,
                   prefix=None)

In [ ]:
with rasterio.open(GRID_IN_F64, 'r') as grid_in_ds, \
        rasterio.open(RASTER_IN, 'r') as array_src_ds, \
        rasterio.open(grid_mask_in_path, "r") as grid_mask_in_ds, \
        rasterio.open(raster_mask_in_path_u8, "r") as raster_mask_in_ds, \
        rasterio.open(output_raster_path, "w", **raster_out_open_args) as array_out_ds, \
        rasterio.open(output_mask_path, "w", **mask_out_open_args) as mask_out_ds:

    basic_grid_resampling_chain(
            grid_ds = grid_in_ds,
            grid_row_coords_band = 1,
            grid_col_coords_band = 2,
            grid_resolution = grid_resolution,
            array_src_ds = array_src_ds,
            array_src_bands = 1,
            array_out_ds = array_out_ds,
            interp = "cubic",
            nodata_out = 0, # <= just to illustrate
            mask_out_ds = mask_out_ds,
            grid_mask_in_ds = grid_mask_in_ds,
            grid_mask_in_unmasked_value = grid_mask_in_valid_value,
            grid_mask_in_band = 1,
        
            array_src_mask_ds = raster_mask_in_ds,
            array_src_mask_band = 1,
            array_src_mask_validity_pair = (array_src_mask_validity_valid,
                                            array_src_mask_validity_invalid),
        )


In [ ]:
with rasterio.open(output_raster_path, "r") as raster_ds, \
        rasterio.open(output_mask_path, "r") as mask_ds:
    print(mask_ds.profile) 
    plot_im({"output image": raster_ds.read(1), "output mask": mask_ds.read(1)})

## Understanding I/O and Memory Management

In the previous section, we worked with rasterio Datasets. As you may have noticed, we did not perform any explicit `read()` or `write()` calls on these Datasets; these operations are handled internally by the `basic_grid_resampling_chain method`.

To give you more control over these I/O operations and memory usage, the method provides several parameters that can be fine-tuned:

- **io_strip_size**
- **io_strip_size_target**
- **tile_shape**
- ncpu (Note: Multiprocessing is not yet implemented for this parameter.)

Adjusting these parameters allows you to manage the memory footprint of your operations.

### The basic_grid_resampling_chain Method's Main Loop

The `basic_grid_resampling_chain` method processes the output raster in sequential line strips. For each strip, it computes all columns (or those defined by an optional window) for a contiguous set of rows. The maximum number of rows in a strip is controlled by the `io_strip_size` parameter.

You can define `io_strip_size` in two distinct modes:

- `GridRIOMode.INPUT`: The target output strip size is calculated by multiplying the` `io_strip_size` value by the input grid's row resolution.

- `GridRIOMode.OUTPUT`: The `io_strip_size` parameter directly defines the target output strip size.

The main loop processes these output strips independently and sequentially:

- Only the required input grid region to compute the current output strip is loaded into memory.

- During each iteration, two single shared buffers are used. These buffers are allocated before the loop to match the larger shape of their respective strips: one for the input grid data and one for the output raster data.

It's important to note that setting the `io_strip_size` parameter to 0 will trigger a unique strip computation, meaning the entire output raster will be processed at once.

### Internal Strip Tiling

The `tile_shape` parameter provides an additional layer of control, enabling a single strip to be computed in smaller tiles. This can further optimize memory usage for very large rasters and mainly impacts the input image raster (and associated mask raster) memory usage efficiency.

## Extended features

### Separate datasets for grid row and columns

As you may have notices, the `basic_grid_resampling_chain` provides two distinct arguments to set the grids dataset, namely `grid_ds` and `grid_col_ds`.

If `grid_col_ds` is set to `None` or 